In [51]:
import pandas as pd
import numpy as np
import random
import math

In [52]:
#Training / Gaussian
def standDeviation(n):
    avg = mean(n)
    var = sum([pow(x-avg,2) for x in n])/float(len(n)-1)
    return math.sqrt(var)

In [53]:
def mean(n):
    return sum(n)/float(len(n))

In [54]:
def normalize(x):
    return (x-mean(x))/ standDeviation(x)

In [55]:
def training(data):
    s = [(mean(a), standDeviation(a)) for a in zip(*data)]
    del s[-1]
    return s

In [56]:
#Testing
def Prob(x, mean, standDeviation):
    e = math.exp(-(math.pow(x-mean,2)/(2*math.pow(standDeviation,2))))
    return (1 / (math.sqrt(2*math.pi) * standDeviation)) * e

In [57]:
def classProb(s, iv):
    prob = {}
    for cv, cs in s.items():
        prob[cv] = 1
        for i in range(len(cs)):
            mean, standDeviation = cs[i]
            x = iv[i]
            prob[cv] *= Prob(x, mean, standDeviation)
    return prob

In [58]:
def p(s, iv):
    prob = classProb(s, iv)
    l, p = None, -1
    for cv, pro in prob.items():
        if l is None or pro > p:
            p = pro
            l = cv
    return l

In [59]:
def prediction(s, test):
    pd = []
    for i in range(len(test)):
        r = p(s, test[i])
        pd.append(r)
    return pd

In [60]:
#main code
df = pd.read_csv('heart.csv')
data = df.to_numpy()
for i in range(len(data)):
    data[i] = [float(x) for x in data[i]]
print(' Heart Dataset loaded\n\n',df)

#Normalize the data
print('Normalize Data: ')
new_data = df.to_numpy()
for i in range(len(data[0])):
    new_data[i] = normalize(data[i])
print('   Data Normalized Done\n\n')

#spliting data for Training and Testing
m = len(new_data)
train_end = int(0.8 * m)
train = data[:train_end]
test = data[train_end:]
print('\nTraining Set(80%) = {0} \nTest Set(20%) = {1}\n\n'.format(len(train), len(test)))

#sumeriesing data (Training part)
print('Training Data: ')
sep = {}
for i in range(len(data)):
    vector = data[i]
    if (vector[-1] not in sep):
        sep[vector[-1]] = []
    sep[vector[-1]].append(vector)
s = {}
for cv, t in sep.items():
    s[cv] = training(t)
print('   ---Training Data is Split ---\n\n')

# testing part (pd)
print(' Testing Data:\n ')
pd = prediction(s, test)
print('   ---Test Data is Split---\n\n')

#confusion matrix
actual = [row[-1] for row in test]
def confusion_matrix(actual, predicted):
    classes       = np.unique(np.concatenate((actual,predicted)))
    confusion_mtx = np.empty((len(classes),len(classes)),dtype=np.int)
    for i,a in enumerate(classes):
        for j,p in enumerate(classes):
            confusion_mtx[i,j] = np.where((actual==a)*(predicted==p))[0].shape[0]
    return confusion_mtx

results = confusion_matrix(actual, pd)

# Calculate Accuracy 
def accuracyCalculation(actual, pd):
    correct = 0
    for x in range(len(actual)):
        if actual[x] == pd[x]:
            correct += 1
    return (correct/float(len(actual)))
accuracy = accuracyCalculation(actual, pd)


print('Confusion Matrix:\n {0}\n'.format(results))

tp=results[0][0]
fp=results[0][1]
fn=results[1][0]
tn=results[1][1]


print('True Positive: {0}'.format(tp))
print('False Positive: {0}'.format(fp))
print('False Negative: {0}'.format(fn))
print('True Negative: {0}'.format(tn))
print('Accuracy: %.3f' % accuracy)
precision = tp / float(fp+tp)
print('Precision: %.3f' % precision)
recall = tn / float(tn+fn)
print('Recall: %.3f' % recall)
fscore = 2*precision*recall / float(precision + recall)
print('Fscore: %.3f' % fscore)




 Heart Dataset loaded

      age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  \
0     63    1   3       145   233    1        0      150      0      2.3   
1     37    1   2       130   250    0        1      187      0      3.5   
2     41    0   1       130   204    0        0      172      0      1.4   
3     56    1   1       120   236    0        1      178      0      0.8   
4     57    0   0       120   354    0        1      163      1      0.6   
..   ...  ...  ..       ...   ...  ...      ...      ...    ...      ...   
298   57    0   0       140   241    0        1      123      1      0.2   
299   45    1   3       110   264    0        1      132      0      1.2   
300   68    1   0       144   193    1        1      141      0      3.4   
301   57    1   0       130   131    0        1      115      1      1.2   
302   57    0   1       130   236    0        0      174      0      0.0   

     slope  ca  thal  target  
0        0   0     1       1  
1

<ipython-input-60-a49d5bfb4f84>:77: RuntimeWarning: invalid value encountered in true_divide
  recall = tn / float(tn+fn)
